# Using Pandas Data Frames to Control ACL Configurations in a Network

## Introduction

ACLs are an essential part of network security, controlling input and output traffic, but as networks today are becoming more and more granular, nearly reaching user level (where traffic flows are defined per endpoint), the amount of acls that a given router must hold becomes increasingly high, and with it the task to manage such configuration becomes herculian.

Using data frames, traffic flows can be configured as a matrix of communication between endpoints, and pandas can be used to process this matrix and produce the resulting acl configurations for each router.

## Sample Network

Lets consider the topology diagram shown below where there are several endpoints distributed across different sites, and connected to each other via a site edge router and a wide area network (WAN).

On the left hand side, there are sites where users connect to, and on the right side there are data centers that host applications that users can connect to. This is a typical enterprise set up, where the user sites are offices and the data centers are the servers hosting the enterprises's applications and data. The connectivity between offices and data centers occurs acros the wan network, but ACLs can be applied at the edge routers of each site to control traffic flow. The allowed traffic will be defined in a matrix containing entries like this:

(A:B)

This means endpoint A allowed to communicate with endpoint B.

The idea is then to group flows in the matrix by source site and destination site, to get the ACLs that need to be configured in each edge router from a central source of information

![title](Topology.png)

## ACL Configuration

Acl configuration has to be configured on the edge routers in both directions to allow traffic from source to destination and viceversa. The reason for this is that traffic flows that are not present in the data frame matrix will be denied.

![title](userIntentDiag.png)

### User site edge router configuration example
```
hostname RUser
!
interface ethernet0
 ip access-group 101 out
!
interface ethernet1
  ip access-group 2 in
!
access-list 101 permit ip 192.168.32.14 0.0.0.0 172.16.8.3 0.0.0.0
access-list 102 permit ip 172.16.8.3 0.0.0.0 192.168.32.14 0.0.0.0
```

### Data center edge router configuration example

```
hostname RDC
!
interface ethernet0
 ip access-group 1 out
!
interface ethernet1
  ip access-group 2 in
!
access-list 101 permit ip 172.16.8.3 0.0.0.0 192.168.32.14 0.0.0.0
access-list 102 permit 192.168.32.14 0.0.0.0 172.16.8.3 0.0.0.0
```

## DataSet Overview

In [5]:
import pandas as pd

dfUsers = pd.read_excel("DataSet.xls", "Users")
dfApps = pd.read_excel("DataSet.xls", "Apps")
dfFlows = pd.read_excel("DataSet.xls", "Rules")

print("Users and where they are located")
print(dfUsers.head())
print("\n")

print("Apps and where they are located")
print(dfApps.head())
print("\n")

print("Allowed traffic flows")
print(dfFlows.head())


Users and where they are located
      Name  Sites      Address
0     Liam     23  192.168.1.1
1     Noah      8  192.168.1.2
2   Oliver     25  192.168.1.3
3  William     40  192.168.1.4
4   Elijah     89  192.168.1.5


Apps and where they are located
         Apps  DataCenters     Address
0    Facebook            2  172.16.8.1
1      Google            1  172.16.8.2
2        Word            5  172.16.8.3
3  PowerPoint            5  172.16.8.4
4       Excel            1  172.16.8.5


Allowed traffic flows
  EndpointA  EndpointB
0    Alaina  CyberDuck
1    Monroe  CyberDuck
2      Cora     Safari
3     Layne  FileZilla
4     Sarah         SM


In [6]:
## Enrich dfFlows with the site data for each endpoint to then be able to group by site
dfUsers.loc[dfUsers['Name'] == 'Thomas']['Sites'].array[0]

def find_user_site(row):
    name = row['EndpointA']
    return dfUsers.loc[dfUsers['Name'] == name]['Sites'].array[0]

def find_app_dc(row):
    app = row['EndpointB']
    return dfApps.loc[dfApps['Apps'] == app]['DataCenters'].array[0]

def find_user_address(row):
    name = row['EndpointA']
    return dfUsers.loc[dfUsers['Name'] == name]['Address'].array[0]

def find_app_address(row):
    app = row['EndpointB']
    return dfApps.loc[dfApps['Apps'] == app]['Address'].array[0]

dfFlows['siteA'] = dfFlows.apply(lambda row: find_user_site(row), axis=1)
dfFlows['siteB'] = dfFlows.apply(lambda row: find_app_dc(row), axis=1)
dfFlows['userAddress'] = dfFlows.apply(lambda row: find_user_address(row), axis=1)
dfFlows['appAddress'] = dfFlows.apply(lambda row: find_app_address(row), axis=1)

print(dfFlows.head())

  EndpointA  EndpointB  siteA  siteB    userAddress   appAddress
0    Alaina  CyberDuck     48      4  192.168.5.174  172.16.8.15
1    Monroe  CyberDuck     20      4   192.168.7.80  172.16.8.15
2      Cora     Safari     88      5   192.168.5.59  172.16.8.19
3     Layne  FileZilla     84      2  192.168.3.177  172.16.8.16
4     Sarah         SM     53      1   192.168.5.69  172.16.8.12


In [7]:
## Group by site
dfFlows.sort_values(['siteA']).head()

,EndpointA,EndpointB,siteA,siteB,userAddress,appAddress
9044,Maria,Safari,1,5,192.168.5.94,172.16.8.19
8963,Adele,Vmware,1,3,192.168.7.245,172.16.8.14
3647,Nehemiah,Atom,1,3,192.168.2.118,172.16.8.18
8928,Kamari,Facebook,1,2,192.168.2.215,172.16.8.1
2274,Adele,CW,1,2,192.168.7.245,172.16.8.13


In [8]:
dfFlows.sort_values(['siteB']).head()

,EndpointA,EndpointB,siteA,siteB,userAddress,appAddress
4999,Remington,Excel,26,1,192.168.1.227,172.16.8.5
1567,Serenity,SM,25,1,192.168.5.67,172.16.8.12
8436,Lauryn,Excel,80,1,192.168.7.124,172.16.8.5
1563,Samuel,SM,70,1,192.168.1.22,172.16.8.12
8437,Reese,Excel,72,1,192.168.3.252,172.16.8.5


In [9]:
from jinja2 import Template

def router_basic_config(site, siteType):
    tmpl = Template(u'''\
    hostname R{{ siteType }}{{ site }}
    !
    interface ethernet0
     ip access-group 101 out
    !
    interface ethernet1
      ip access-group 2 in
    !
    ''')
    
    config = tmpl.render(
        site=u"{}".format(site),
        siteType=u"{}".format(siteType)
    )
    
    print(config)

def acl(source, destination, seqnumber):
    tmpl = Template(u'''\
    access-list 101 {{ seqnumber }} permit ip {{ source }} 0.0.0.0 {{ destination }} 0.0.0.0
    access-list 102 {{ seqnumber }} permit ip {{ destination }} 0.0.0.0 {{ source }} 0.0.0.0
    ''')
    
    config = tmpl.render(
        source = u"{}".format(source),
        destination = u"{}".format(destination),
        seqnumber = u"{}".format(seqnumber)
    )
    
    print(config)

In [10]:
for i, group in dfFlows.groupby(['siteA']):
    router_basic_config(group.iloc[1]['siteA'], "user")
    seqnumber=5
    for entry in group[['userAddress', 'appAddress']].itertuples():
        seqnumber+=5
        acl(entry[1], entry[2],seqnumber)

    hostname Ruser1
    !
    interface ethernet0
     ip access-group 101 out
    !
    interface ethernet1
      ip access-group 2 in
    !
    
    access-list 101 10 permit ip 192.168.5.94 0.0.0.0 172.16.8.15 0.0.0.0
    access-list 102 10 permit ip 172.16.8.15 0.0.0.0 192.168.5.94 0.0.0.0
    
    access-list 101 15 permit ip 192.168.6.243 0.0.0.0 172.16.8.11 0.0.0.0
    access-list 102 15 permit ip 172.16.8.11 0.0.0.0 192.168.6.243 0.0.0.0
    
    access-list 101 20 permit ip 192.168.1.53 0.0.0.0 172.16.8.7 0.0.0.0
    access-list 102 20 permit ip 172.16.8.7 0.0.0.0 192.168.1.53 0.0.0.0
    
    access-list 101 25 permit ip 192.168.5.231 0.0.0.0 172.16.8.8 0.0.0.0
    access-list 102 25 permit ip 172.16.8.8 0.0.0.0 192.168.5.231 0.0.0.0
    
    access-list 101 30 permit ip 192.168.2.215 0.0.0.0 172.16.8.16 0.0.0.0
    access-list 102 30 permit ip 172.16.8.16 0.0.0.0 192.168.2.215 0.0.0.0
    
    access-list 101 35 permit ip 192.168.5.225 0.0.0.0 172.16.8.10 0.0.0.0
    access-

    
    access-list 101 215 permit ip 192.168.7.27 0.0.0.0 172.16.8.7 0.0.0.0
    access-list 102 215 permit ip 172.16.8.7 0.0.0.0 192.168.7.27 0.0.0.0
    
    access-list 101 220 permit ip 192.168.4.105 0.0.0.0 172.16.8.12 0.0.0.0
    access-list 102 220 permit ip 172.16.8.12 0.0.0.0 192.168.4.105 0.0.0.0
    
    access-list 101 225 permit ip 192.168.2.130 0.0.0.0 172.16.8.12 0.0.0.0
    access-list 102 225 permit ip 172.16.8.12 0.0.0.0 192.168.2.130 0.0.0.0
    
    access-list 101 230 permit ip 192.168.7.117 0.0.0.0 172.16.8.7 0.0.0.0
    access-list 102 230 permit ip 172.16.8.7 0.0.0.0 192.168.7.117 0.0.0.0
    
    access-list 101 235 permit ip 192.168.2.69 0.0.0.0 172.16.8.2 0.0.0.0
    access-list 102 235 permit ip 172.16.8.2 0.0.0.0 192.168.2.69 0.0.0.0
    
    access-list 101 240 permit ip 192.168.2.69 0.0.0.0 172.16.8.15 0.0.0.0
    access-list 102 240 permit ip 172.16.8.15 0.0.0.0 192.168.2.69 0.0.0.0
    
    access-list 101 245 permit ip 192.168.4.95 0.0.0.0 172.16.8.1

    
    access-list 101 545 permit ip 192.168.5.220 0.0.0.0 172.16.8.11 0.0.0.0
    access-list 102 545 permit ip 172.16.8.11 0.0.0.0 192.168.5.220 0.0.0.0
    
    access-list 101 550 permit ip 192.168.5.220 0.0.0.0 172.16.8.2 0.0.0.0
    access-list 102 550 permit ip 172.16.8.2 0.0.0.0 192.168.5.220 0.0.0.0
    
    hostname Ruser3
    !
    interface ethernet0
     ip access-group 101 out
    !
    interface ethernet1
      ip access-group 2 in
    !
    
    access-list 101 10 permit ip 192.168.6.188 0.0.0.0 172.16.8.14 0.0.0.0
    access-list 102 10 permit ip 172.16.8.14 0.0.0.0 192.168.6.188 0.0.0.0
    
    access-list 101 15 permit ip 192.168.1.237 0.0.0.0 172.16.8.17 0.0.0.0
    access-list 102 15 permit ip 172.16.8.17 0.0.0.0 192.168.1.237 0.0.0.0
    
    access-list 101 20 permit ip 192.168.6.188 0.0.0.0 172.16.8.15 0.0.0.0
    access-list 102 20 permit ip 172.16.8.15 0.0.0.0 192.168.6.188 0.0.0.0
    
    access-list 101 25 permit ip 192.168.3.101 0.0.0.0 172.16.8.12 0.0.

    
    access-list 101 145 permit ip 192.168.2.78 0.0.0.0 172.16.8.19 0.0.0.0
    access-list 102 145 permit ip 172.16.8.19 0.0.0.0 192.168.2.78 0.0.0.0
    
    access-list 101 150 permit ip 192.168.2.51 0.0.0.0 172.16.8.18 0.0.0.0
    access-list 102 150 permit ip 172.16.8.18 0.0.0.0 192.168.2.51 0.0.0.0
    
    access-list 101 155 permit ip 192.168.4.129 0.0.0.0 172.16.8.2 0.0.0.0
    access-list 102 155 permit ip 172.16.8.2 0.0.0.0 192.168.4.129 0.0.0.0
    
    access-list 101 160 permit ip 192.168.3.150 0.0.0.0 172.16.8.1 0.0.0.0
    access-list 102 160 permit ip 172.16.8.1 0.0.0.0 192.168.3.150 0.0.0.0
    
    access-list 101 165 permit ip 192.168.4.14 0.0.0.0 172.16.8.18 0.0.0.0
    access-list 102 165 permit ip 172.16.8.18 0.0.0.0 192.168.4.14 0.0.0.0
    
    access-list 101 170 permit ip 192.168.4.14 0.0.0.0 172.16.8.18 0.0.0.0
    access-list 102 170 permit ip 172.16.8.18 0.0.0.0 192.168.4.14 0.0.0.0
    
    access-list 101 175 permit ip 192.168.8.192 0.0.0.0 172.16.8.

    
    access-list 101 120 permit ip 192.168.3.191 0.0.0.0 172.16.8.8 0.0.0.0
    access-list 102 120 permit ip 172.16.8.8 0.0.0.0 192.168.3.191 0.0.0.0
    
    access-list 101 125 permit ip 192.168.8.49 0.0.0.0 172.16.8.9 0.0.0.0
    access-list 102 125 permit ip 172.16.8.9 0.0.0.0 192.168.8.49 0.0.0.0
    
    access-list 101 130 permit ip 192.168.3.205 0.0.0.0 172.16.8.7 0.0.0.0
    access-list 102 130 permit ip 172.16.8.7 0.0.0.0 192.168.3.205 0.0.0.0
    
    access-list 101 135 permit ip 192.168.4.56 0.0.0.0 172.16.8.13 0.0.0.0
    access-list 102 135 permit ip 172.16.8.13 0.0.0.0 192.168.4.56 0.0.0.0
    
    access-list 101 140 permit ip 192.168.3.205 0.0.0.0 172.16.8.19 0.0.0.0
    access-list 102 140 permit ip 172.16.8.19 0.0.0.0 192.168.3.205 0.0.0.0
    
    access-list 101 145 permit ip 192.168.2.168 0.0.0.0 172.16.8.8 0.0.0.0
    access-list 102 145 permit ip 172.16.8.8 0.0.0.0 192.168.2.168 0.0.0.0
    
    access-list 101 150 permit ip 192.168.2.9 0.0.0.0 172.16.8.8 

...

    
    access-list 101 350 permit ip 192.168.4.146 0.0.0.0 172.16.8.15 0.0.0.0
    access-list 102 350 permit ip 172.16.8.15 0.0.0.0 192.168.4.146 0.0.0.0
    
    access-list 101 355 permit ip 192.168.4.25 0.0.0.0 172.16.8.10 0.0.0.0
    access-list 102 355 permit ip 172.16.8.10 0.0.0.0 192.168.4.25 0.0.0.0
    
    access-list 101 360 permit ip 192.168.8.173 0.0.0.0 172.16.8.19 0.0.0.0
    access-list 102 360 permit ip 172.16.8.19 0.0.0.0 192.168.8.173 0.0.0.0
    
    access-list 101 365 permit ip 192.168.4.146 0.0.0.0 172.16.8.13 0.0.0.0
    access-list 102 365 permit ip 172.16.8.13 0.0.0.0 192.168.4.146 0.0.0.0
    
    access-list 101 370 permit ip 192.168.6.113 0.0.0.0 172.16.8.10 0.0.0.0
    access-list 102 370 permit ip 172.16.8.10 0.0.0.0 192.168.6.113 0.0.0.0
    
    access-list 101 375 permit ip 192.168.3.221 0.0.0.0 172.16.8.18 0.0.0.0
    access-list 102 375 permit ip 172.16.8.18 0.0.0.0 192.168.3.221 0.0.0.0
    
    access-list 101 380 permit ip 192.168.1.40 0.0.0.0 

    
    access-list 101 260 permit ip 192.168.6.163 0.0.0.0 172.16.8.14 0.0.0.0
    access-list 102 260 permit ip 172.16.8.14 0.0.0.0 192.168.6.163 0.0.0.0
    
    access-list 101 265 permit ip 192.168.1.23 0.0.0.0 172.16.8.10 0.0.0.0
    access-list 102 265 permit ip 172.16.8.10 0.0.0.0 192.168.1.23 0.0.0.0
    
    access-list 101 270 permit ip 192.168.3.153 0.0.0.0 172.16.8.5 0.0.0.0
    access-list 102 270 permit ip 172.16.8.5 0.0.0.0 192.168.3.153 0.0.0.0
    
    access-list 101 275 permit ip 192.168.8.106 0.0.0.0 172.16.8.5 0.0.0.0
    access-list 102 275 permit ip 172.16.8.5 0.0.0.0 192.168.8.106 0.0.0.0
    
    access-list 101 280 permit ip 192.168.3.57 0.0.0.0 172.16.8.8 0.0.0.0
    access-list 102 280 permit ip 172.16.8.8 0.0.0.0 192.168.3.57 0.0.0.0
    
    access-list 101 285 permit ip 192.168.4.11 0.0.0.0 172.16.8.2 0.0.0.0
    access-list 102 285 permit ip 172.16.8.2 0.0.0.0 192.168.4.11 0.0.0.0
    
    access-list 101 290 permit ip 192.168.5.65 0.0.0.0 172.16.8.13 

    
    access-list 101 365 permit ip 192.168.1.164 0.0.0.0 172.16.8.7 0.0.0.0
    access-list 102 365 permit ip 172.16.8.7 0.0.0.0 192.168.1.164 0.0.0.0
    
    access-list 101 370 permit ip 192.168.6.187 0.0.0.0 172.16.8.1 0.0.0.0
    access-list 102 370 permit ip 172.16.8.1 0.0.0.0 192.168.6.187 0.0.0.0
    
    access-list 101 375 permit ip 192.168.4.136 0.0.0.0 172.16.8.9 0.0.0.0
    access-list 102 375 permit ip 172.16.8.9 0.0.0.0 192.168.4.136 0.0.0.0
    
    access-list 101 380 permit ip 192.168.1.169 0.0.0.0 172.16.8.8 0.0.0.0
    access-list 102 380 permit ip 172.16.8.8 0.0.0.0 192.168.1.169 0.0.0.0
    
    access-list 101 385 permit ip 192.168.3.143 0.0.0.0 172.16.8.6 0.0.0.0
    access-list 102 385 permit ip 172.16.8.6 0.0.0.0 192.168.3.143 0.0.0.0
    
    access-list 101 390 permit ip 192.168.7.125 0.0.0.0 172.16.8.14 0.0.0.0
    access-list 102 390 permit ip 172.16.8.14 0.0.0.0 192.168.7.125 0.0.0.0
    
    access-list 101 395 permit ip 192.168.7.175 0.0.0.0 172.16.

    
    access-list 101 430 permit ip 192.168.1.24 0.0.0.0 172.16.8.1 0.0.0.0
    access-list 102 430 permit ip 172.16.8.1 0.0.0.0 192.168.1.24 0.0.0.0
    
    access-list 101 435 permit ip 192.168.2.88 0.0.0.0 172.16.8.4 0.0.0.0
    access-list 102 435 permit ip 172.16.8.4 0.0.0.0 192.168.2.88 0.0.0.0
    
    access-list 101 440 permit ip 192.168.1.196 0.0.0.0 172.16.8.7 0.0.0.0
    access-list 102 440 permit ip 172.16.8.7 0.0.0.0 192.168.1.196 0.0.0.0
    
    access-list 101 445 permit ip 192.168.3.15 0.0.0.0 172.16.8.3 0.0.0.0
    access-list 102 445 permit ip 172.16.8.3 0.0.0.0 192.168.3.15 0.0.0.0
    
    access-list 101 450 permit ip 192.168.5.14 0.0.0.0 172.16.8.11 0.0.0.0
    access-list 102 450 permit ip 172.16.8.11 0.0.0.0 192.168.5.14 0.0.0.0
    
    access-list 101 455 permit ip 192.168.4.64 0.0.0.0 172.16.8.1 0.0.0.0
    access-list 102 455 permit ip 172.16.8.1 0.0.0.0 192.168.4.64 0.0.0.0
    
    access-list 101 460 permit ip 192.168.3.22 0.0.0.0 172.16.8.13 0.0.0.

    
    access-list 101 280 permit ip 192.168.5.23 0.0.0.0 172.16.8.7 0.0.0.0
    access-list 102 280 permit ip 172.16.8.7 0.0.0.0 192.168.5.23 0.0.0.0
    
    access-list 101 285 permit ip 192.168.3.96 0.0.0.0 172.16.8.1 0.0.0.0
    access-list 102 285 permit ip 172.16.8.1 0.0.0.0 192.168.3.96 0.0.0.0
    
    access-list 101 290 permit ip 192.168.8.177 0.0.0.0 172.16.8.2 0.0.0.0
    access-list 102 290 permit ip 172.16.8.2 0.0.0.0 192.168.8.177 0.0.0.0
    
    access-list 101 295 permit ip 192.168.8.101 0.0.0.0 172.16.8.8 0.0.0.0
    access-list 102 295 permit ip 172.16.8.8 0.0.0.0 192.168.8.101 0.0.0.0
    
    access-list 101 300 permit ip 192.168.1.132 0.0.0.0 172.16.8.4 0.0.0.0
    access-list 102 300 permit ip 172.16.8.4 0.0.0.0 192.168.1.132 0.0.0.0
    
    access-list 101 305 permit ip 192.168.6.126 0.0.0.0 172.16.8.2 0.0.0.0
    access-list 102 305 permit ip 172.16.8.2 0.0.0.0 192.168.6.126 0.0.0.0
    
    access-list 101 310 permit ip 192.168.7.141 0.0.0.0 172.16.8.3 0.

In [11]:
for i, group in dfFlows.groupby(['siteB']):
    router_basic_config(group.iloc[1]['siteB'], "DC")
    seqnumber=5
    for entry in group[['appAddress', 'userAddress']].itertuples():
        seqnumber += 5
        acl(entry[1], entry[2], seqnumber)

    hostname RDC1
    !
    interface ethernet0
     ip access-group 101 out
    !
    interface ethernet1
      ip access-group 2 in
    !
    
    access-list 101 10 permit ip 172.16.8.12 0.0.0.0 192.168.5.69 0.0.0.0
    access-list 102 10 permit ip 192.168.5.69 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 15 permit ip 172.16.8.5 0.0.0.0 192.168.2.18 0.0.0.0
    access-list 102 15 permit ip 192.168.2.18 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 20 permit ip 172.16.8.5 0.0.0.0 192.168.3.26 0.0.0.0
    access-list 102 20 permit ip 192.168.3.26 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 25 permit ip 172.16.8.5 0.0.0.0 192.168.8.7 0.0.0.0
    access-list 102 25 permit ip 192.168.8.7 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 30 permit ip 172.16.8.2 0.0.0.0 192.168.7.65 0.0.0.0
    access-list 102 30 permit ip 192.168.7.65 0.0.0.0 172.16.8.2 0.0.0.0
    
    access-list 101 35 permit ip 172.16.8.2 0.0.0.0 192.168.5.222 0.0.0.0
    access-list 102 35 per

    
    access-list 101 600 permit ip 172.16.8.2 0.0.0.0 192.168.4.77 0.0.0.0
    access-list 102 600 permit ip 192.168.4.77 0.0.0.0 172.16.8.2 0.0.0.0
    
    access-list 101 605 permit ip 172.16.8.12 0.0.0.0 192.168.8.202 0.0.0.0
    access-list 102 605 permit ip 192.168.8.202 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 610 permit ip 172.16.8.5 0.0.0.0 192.168.1.126 0.0.0.0
    access-list 102 610 permit ip 192.168.1.126 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 615 permit ip 172.16.8.12 0.0.0.0 192.168.5.55 0.0.0.0
    access-list 102 615 permit ip 192.168.5.55 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 620 permit ip 172.16.8.5 0.0.0.0 192.168.8.127 0.0.0.0
    access-list 102 620 permit ip 192.168.8.127 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 625 permit ip 172.16.8.2 0.0.0.0 192.168.6.127 0.0.0.0
    access-list 102 625 permit ip 192.168.6.127 0.0.0.0 172.16.8.2 0.0.0.0
    
    access-list 101 630 permit ip 172.16.8.12 0.0.0.0 192.168.4.1

    
    access-list 101 965 permit ip 172.16.8.5 0.0.0.0 192.168.2.140 0.0.0.0
    access-list 102 965 permit ip 192.168.2.140 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 970 permit ip 172.16.8.12 0.0.0.0 192.168.3.33 0.0.0.0
    access-list 102 970 permit ip 192.168.3.33 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 975 permit ip 172.16.8.5 0.0.0.0 192.168.4.98 0.0.0.0
    access-list 102 975 permit ip 192.168.4.98 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 980 permit ip 172.16.8.5 0.0.0.0 192.168.7.226 0.0.0.0
    access-list 102 980 permit ip 192.168.7.226 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 985 permit ip 172.16.8.5 0.0.0.0 192.168.5.159 0.0.0.0
    access-list 102 985 permit ip 192.168.5.159 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 990 permit ip 172.16.8.12 0.0.0.0 192.168.8.207 0.0.0.0
    access-list 102 990 permit ip 192.168.8.207 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 995 permit ip 172.16.8.5 0.0.0.0 192.168.2.25

    
    access-list 101 1575 permit ip 172.16.8.12 0.0.0.0 192.168.2.28 0.0.0.0
    access-list 102 1575 permit ip 192.168.2.28 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 1580 permit ip 172.16.8.12 0.0.0.0 192.168.8.171 0.0.0.0
    access-list 102 1580 permit ip 192.168.8.171 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 1585 permit ip 172.16.8.12 0.0.0.0 192.168.8.205 0.0.0.0
    access-list 102 1585 permit ip 192.168.8.205 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 1590 permit ip 172.16.8.12 0.0.0.0 192.168.1.187 0.0.0.0
    access-list 102 1590 permit ip 192.168.1.187 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 1595 permit ip 172.16.8.12 0.0.0.0 192.168.6.196 0.0.0.0
    access-list 102 1595 permit ip 192.168.6.196 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 1600 permit ip 172.16.8.12 0.0.0.0 192.168.6.203 0.0.0.0
    access-list 102 1600 permit ip 192.168.6.203 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 1605 permit ip 172.16.8

    
    access-list 101 1985 permit ip 172.16.8.5 0.0.0.0 192.168.4.10 0.0.0.0
    access-list 102 1985 permit ip 192.168.4.10 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 1990 permit ip 172.16.8.5 0.0.0.0 192.168.4.141 0.0.0.0
    access-list 102 1990 permit ip 192.168.4.141 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 1995 permit ip 172.16.8.12 0.0.0.0 192.168.5.4 0.0.0.0
    access-list 102 1995 permit ip 192.168.5.4 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 2000 permit ip 172.16.8.12 0.0.0.0 192.168.5.126 0.0.0.0
    access-list 102 2000 permit ip 192.168.5.126 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 2005 permit ip 172.16.8.12 0.0.0.0 192.168.5.1 0.0.0.0
    access-list 102 2005 permit ip 192.168.5.1 0.0.0.0 172.16.8.12 0.0.0.0
    
    access-list 101 2010 permit ip 172.16.8.5 0.0.0.0 192.168.5.60 0.0.0.0
    access-list 102 2010 permit ip 192.168.5.60 0.0.0.0 172.16.8.5 0.0.0.0
    
    access-list 101 2015 permit ip 172.16.8.2 0.0.0.0 192.1

...

    
    access-list 101 16285 permit ip 172.16.8.10 0.0.0.0 192.168.4.163 0.0.0.0
    access-list 102 16285 permit ip 192.168.4.163 0.0.0.0 172.16.8.10 0.0.0.0
    
    access-list 101 16290 permit ip 172.16.8.4 0.0.0.0 192.168.4.111 0.0.0.0
    access-list 102 16290 permit ip 192.168.4.111 0.0.0.0 172.16.8.4 0.0.0.0
    
    access-list 101 16295 permit ip 172.16.8.11 0.0.0.0 192.168.3.53 0.0.0.0
    access-list 102 16295 permit ip 192.168.3.53 0.0.0.0 172.16.8.11 0.0.0.0
    
    access-list 101 16300 permit ip 172.16.8.9 0.0.0.0 192.168.7.69 0.0.0.0
    access-list 102 16300 permit ip 192.168.7.69 0.0.0.0 172.16.8.9 0.0.0.0
    
    access-list 101 16305 permit ip 172.16.8.10 0.0.0.0 192.168.5.165 0.0.0.0
    access-list 102 16305 permit ip 192.168.5.165 0.0.0.0 172.16.8.10 0.0.0.0
    
    access-list 101 16310 permit ip 172.16.8.8 0.0.0.0 192.168.6.247 0.0.0.0
    access-list 102 16310 permit ip 192.168.6.247 0.0.0.0 172.16.8.8 0.0.0.0
    
    access-list 101 16315 permit ip 172

    
    access-list 101 16555 permit ip 172.16.8.9 0.0.0.0 192.168.5.168 0.0.0.0
    access-list 102 16555 permit ip 192.168.5.168 0.0.0.0 172.16.8.9 0.0.0.0
    
    access-list 101 16560 permit ip 172.16.8.11 0.0.0.0 192.168.6.235 0.0.0.0
    access-list 102 16560 permit ip 192.168.6.235 0.0.0.0 172.16.8.11 0.0.0.0
    
    access-list 101 16565 permit ip 172.16.8.9 0.0.0.0 192.168.8.133 0.0.0.0
    access-list 102 16565 permit ip 192.168.8.133 0.0.0.0 172.16.8.9 0.0.0.0
    
    access-list 101 16570 permit ip 172.16.8.19 0.0.0.0 192.168.1.128 0.0.0.0
    access-list 102 16570 permit ip 192.168.1.128 0.0.0.0 172.16.8.19 0.0.0.0
    
    access-list 101 16575 permit ip 172.16.8.11 0.0.0.0 192.168.6.79 0.0.0.0
    access-list 102 16575 permit ip 192.168.6.79 0.0.0.0 172.16.8.11 0.0.0.0
    
    access-list 101 16580 permit ip 172.16.8.10 0.0.0.0 192.168.1.86 0.0.0.0
    access-list 102 16580 permit ip 192.168.1.86 0.0.0.0 172.16.8.10 0.0.0.0
    
    access-list 101 16585 permit ip 1

    
    access-list 101 17125 permit ip 172.16.8.9 0.0.0.0 192.168.7.191 0.0.0.0
    access-list 102 17125 permit ip 192.168.7.191 0.0.0.0 172.16.8.9 0.0.0.0
    
    access-list 101 17130 permit ip 172.16.8.3 0.0.0.0 192.168.6.186 0.0.0.0
    access-list 102 17130 permit ip 192.168.6.186 0.0.0.0 172.16.8.3 0.0.0.0
    
    access-list 101 17135 permit ip 172.16.8.11 0.0.0.0 192.168.5.21 0.0.0.0
    access-list 102 17135 permit ip 192.168.5.21 0.0.0.0 172.16.8.11 0.0.0.0
    
    access-list 101 17140 permit ip 172.16.8.9 0.0.0.0 192.168.4.96 0.0.0.0
    access-list 102 17140 permit ip 192.168.4.96 0.0.0.0 172.16.8.9 0.0.0.0
    
    access-list 101 17145 permit ip 172.16.8.9 0.0.0.0 192.168.4.139 0.0.0.0
    access-list 102 17145 permit ip 192.168.4.139 0.0.0.0 172.16.8.9 0.0.0.0
    
    access-list 101 17150 permit ip 172.16.8.19 0.0.0.0 192.168.2.67 0.0.0.0
    access-list 102 17150 permit ip 192.168.2.67 0.0.0.0 172.16.8.19 0.0.0.0
    
    access-list 101 17155 permit ip 172.16.

    
    access-list 101 17465 permit ip 172.16.8.10 0.0.0.0 192.168.7.137 0.0.0.0
    access-list 102 17465 permit ip 192.168.7.137 0.0.0.0 172.16.8.10 0.0.0.0
    
    access-list 101 17470 permit ip 172.16.8.10 0.0.0.0 192.168.4.73 0.0.0.0
    access-list 102 17470 permit ip 192.168.4.73 0.0.0.0 172.16.8.10 0.0.0.0
    
    access-list 101 17475 permit ip 172.16.8.10 0.0.0.0 192.168.6.33 0.0.0.0
    access-list 102 17475 permit ip 192.168.6.33 0.0.0.0 172.16.8.10 0.0.0.0
    
    access-list 101 17480 permit ip 172.16.8.10 0.0.0.0 192.168.8.110 0.0.0.0
    access-list 102 17480 permit ip 192.168.8.110 0.0.0.0 172.16.8.10 0.0.0.0
    
    access-list 101 17485 permit ip 172.16.8.10 0.0.0.0 192.168.7.110 0.0.0.0
    access-list 102 17485 permit ip 192.168.7.110 0.0.0.0 172.16.8.10 0.0.0.0
    
    access-list 101 17490 permit ip 172.16.8.4 0.0.0.0 192.168.1.94 0.0.0.0
    access-list 102 17490 permit ip 192.168.1.94 0.0.0.0 172.16.8.4 0.0.0.0
    
    access-list 101 17495 permit ip 1

    
    access-list 101 18110 permit ip 172.16.8.4 0.0.0.0 192.168.5.4 0.0.0.0
    access-list 102 18110 permit ip 192.168.5.4 0.0.0.0 172.16.8.4 0.0.0.0
    
    access-list 101 18115 permit ip 172.16.8.11 0.0.0.0 192.168.7.84 0.0.0.0
    access-list 102 18115 permit ip 192.168.7.84 0.0.0.0 172.16.8.11 0.0.0.0
    
    access-list 101 18120 permit ip 172.16.8.19 0.0.0.0 192.168.4.52 0.0.0.0
    access-list 102 18120 permit ip 192.168.4.52 0.0.0.0 172.16.8.19 0.0.0.0
    
    access-list 101 18125 permit ip 172.16.8.4 0.0.0.0 192.168.3.132 0.0.0.0
    access-list 102 18125 permit ip 192.168.3.132 0.0.0.0 172.16.8.4 0.0.0.0
    
    access-list 101 18130 permit ip 172.16.8.3 0.0.0.0 192.168.3.92 0.0.0.0
    access-list 102 18130 permit ip 192.168.3.92 0.0.0.0 172.16.8.3 0.0.0.0
    
    access-list 101 18135 permit ip 172.16.8.3 0.0.0.0 192.168.7.235 0.0.0.0
    access-list 102 18135 permit ip 192.168.7.235 0.0.0.0 172.16.8.3 0.0.0.0
    
    access-list 101 18140 permit ip 172.16.8.4 